In [1]:
import os
import numpy as np
import pandas as pd
from numba import jit
from tqdm import tqdm
import statsmodels.api as sm
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import Alpha as alpha

import warnings
warnings.filterwarnings('ignore')

# pmCO_pmSV

In [23]:
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\close")
freeshare = alpha.load_data('pv\\stock_shareout')
sv = []
co = []

for date in tqdm(data):
    volume_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\volume",date)
    open_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\open",date)
    close_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\close",date)
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    volume = pd.read_parquet(volume_path).loc[130000:]
    opn = pd.read_parquet(open_path).loc[130000:]
    close = pd.read_parquet(close_path).loc[130000:]
    code = close.columns

    pmCO = np.subtract(close.iloc[-1],opn.iloc[0])
    pmCO.name = date
    smart_t = (abs(np.divide(close,opn)-1)/np.sqrt(volume)).rank(axis=0,pct=True,ascending=False)
    pmSV = np.nansum(np.where(smart_t<=0.2, volume, np.nan),axis=0)
    pmSV = pd.Series(pmSV,index=code,name=date)
    
    sv.append(pmSV)
    co.append(pmCO)

pmSV = pd.concat(sv,axis=1).T
pmCO = pd.concat(co,axis=1).T

pmSV = pmSV.div(freeshare,axis=0).dropna(axis=0,how='all')

pmCO_pmSV = alpha.mvind_neutral(pmSV.rolling(20).corr(pmCO.loc[pmSV.index]).iloc[19:].dropna(axis=0,how='all'))

100%|██████████████████████████████████████████████████████████████████████████████| 2473/2473 [00:38<00:00, 63.89it/s]


# SCOV

In [12]:
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\volume")
freeshare = alpha.load_data('pv\\stock_shareout')
res = []
for date in tqdm(data):
    volume_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\volume",date)
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    volume = pd.read_parquet(volume_path).loc[143000:].sum()
    
    res.append(volume)
Y1430V = pd.concat(res,axis=1).T
Y1430V.index = [pd.to_datetime(date.split('.')[0],format='%Y%m%d') for date in data]
Y1430V = Y1430V.div(freeshare,axis=0).shift(1).dropna(axis=0,how='all')

close = alpha.load_data('pv\\stock_close')
opn = alpha.load_data('pv\\stock_open')
OYC = opn.sub(close.shift(1))

SCOV = alpha.mvind_neutral(OYC.rolling(20).corr(Y1430V).iloc[19:].dropna(axis=0,how='all'))

100%|██████████████████████████████████████████████████████████████████████████████| 2472/2472 [00:38<00:00, 63.41it/s]


# SCOV

In [24]:
SRV = (alpha.zscore(pmCO_pmSV)-alpha.zscore(SCOV)).dropna(axis=0,how='all')

In [25]:
alpha.to_fctDataBase(SRV.dropna(axis=0,how='all'),'SRV')

In [27]:
alpha.Alpha(SRV, 'SRV', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\alpha_result\report\SRV_W.xlsx
